# Practice 99 — MLP for Predictive Maintenance

Practice 06 (Extra)에서 선형 모델로 풀기 어려웠던 **제조 설비 고장 예측** 문제를
**다층 퍼셉트론(MLP)**으로 해결합니다.

**AI4I 2020 Predictive Maintenance Dataset** ([UCI #601](https://archive.ics.uci.edu/dataset/601/ai4i+2020+predictive+maintenance+dataset))

| # | Model | Features | Expected Accuracy |
|---|---|---|---|
| 1 | Logistic Regression (NumPy) | 2개 (RPM, Torque) | ~75% |
| 2 | Logistic Regression (NumPy) | 5개 (전체) | ~83% |
| 3 | MLP (PyTorch) | 5개 (전체) | **~96%** |

> **핵심 메시지:**  
> 선형 모델 + 소수 변수로는 한계가 있는 산업 데이터에서,  
> **더 많은 변수 + 비선형 모델(MLP)**이 정확도를 극적으로 향상시킵니다.

In [ ]:
%pip install -q ucimlrepo

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo

plt.rcParams['figure.figsize'] = (10, 4)
plt.rcParams['axes.unicode_minus'] = False

---
# 데이터 로드 — AI4I 2020 Predictive Maintenance

제조 설비의 센서 데이터로 **고장 여부**를 예측하는 이진 분류 문제입니다.

| Feature | 설명 | 단위 |
|---|---|---|
| **Air temperature** | 공기 온도 | K |
| **Process temperature** | 공정 온도 | K |
| **Rotational speed** | 회전 속도 | rpm |
| **Torque** | 토크 | Nm |
| **Tool wear** | 공구 마모도 | min |

> **기계공학과의 관련성:**  
> 회전 기계의 진동, 온도, 토크 등 센서 데이터로 고장을 사전에 예측하는 것은  
> **예지 정비(Predictive Maintenance)**의 핵심이며, 스마트 제조의 기반 기술입니다.

### Class Imbalance 문제

원본 데이터는 고장 비율이 ~3.4%로 극도로 불균형합니다.  
불균형 데이터에서 "전부 정상"이라고 예측해도 96.6% 정확도가 나오므로,  
**균형 샘플링(undersampling)**으로 정상:고장 = 1:1로 맞춥니다.

In [ ]:
# AI4I 2020 Predictive Maintenance (UCI #601)
ai4i = fetch_ucirepo(id=601)
X_all = ai4i.data.features
y_all = ai4i.data.targets

print('=== Raw Data ===')
print(f'Features: {X_all.columns.tolist()}')
print(f'Targets:  {y_all.columns.tolist()}')
print(f'Shape:    {X_all.shape}')

In [ ]:
# 수치형 5개 feature 추출
feature_cols = ['Air temperature', 'Process temperature',
                'Rotational speed', 'Torque', 'Tool wear']
X_raw = X_all[feature_cols].values.astype(float)
y_raw = y_all['Machine failure'].values.astype(int)

print(f'정상: {(y_raw==0).sum()}, 고장: {(y_raw==1).sum()} ({y_raw.mean():.1%} failure rate)')

# --- 균형 샘플링 (undersampling) ---
np.random.seed(42)
idx_fail = np.where(y_raw == 1)[0]                         # 고장 전체
idx_normal = np.where(y_raw == 0)[0]
idx_normal_sub = np.random.choice(idx_normal, len(idx_fail), replace=False)
idx = np.concatenate([idx_fail, idx_normal_sub])
np.random.shuffle(idx)

X_bal = X_raw[idx]
y_bal = y_raw[idx]

print(f'\n=== Balanced Data ===')
print(f'정상: {(y_bal==0).sum()}, 고장: {(y_bal==1).sum()}, 총: {len(y_bal)}')

---
# 1. Logistic Regression — 2 Features

Practice 06에서 사용한 것과 동일한 **NumPy gradient descent** 코드로  
**회전 속도(RPM)**와 **토크(Torque)** 2개 변수만 사용합니다.

$$\sigma(z) = \frac{1}{1+e^{-z}}, \qquad J = -\sum_n \bigl[ y_n \log \sigma(z_n) + (1-y_n) \log(1-\sigma(z_n)) \bigr]$$

In [ ]:
# 2 features: RPM + Torque
f1_raw = X_bal[:, 2]   # Rotational speed
f2_raw = X_bal[:, 3]   # Torque

# 표준화
f1_mean, f1_std = f1_raw.mean(), f1_raw.std()
f2_mean, f2_std = f2_raw.mean(), f2_raw.std()
f1 = (f1_raw - f1_mean) / f1_std
f2 = (f2_raw - f2_mean) / f2_std

X2 = np.column_stack([np.ones(len(y_bal)), f1, f2])

sigmoid = lambda z: 1 / (1 + np.exp(-np.clip(z, -500, 500)))

w = np.zeros(3)
rho = 0.01
n_epochs = 300

loss_hist_2f = []
for ep in range(n_epochs):
    o = sigmoid(X2 @ w)
    loss = -np.sum(y_bal * np.log(o+1e-15) + (1-y_bal) * np.log(1-o+1e-15))
    loss_hist_2f.append(loss)
    grad = X2.T @ (o - y_bal)
    w = w - rho * grad

pred_2f = (sigmoid(X2 @ w) >= 0.5).astype(int)
acc_2f = np.mean(pred_2f == y_bal)
print(f'Logistic (2 features) Accuracy: {acc_2f:.1%}')

In [ ]:
fig, axes = plt.subplots(1, 2)
c0, c1 = y_bal==0, y_bal==1

axes[0].scatter(f1_raw[c0], f2_raw[c0], c='b', s=10, alpha=0.3, label='Normal')
axes[0].scatter(f1_raw[c1], f2_raw[c1], c='r', s=10, alpha=0.3, label='Failure')

# 결정 경계
if w[2] != 0:
    x1_line = np.linspace(f1_raw.min(), f1_raw.max(), 100)
    x1n = (x1_line - f1_mean) / f1_std
    x2n = -(w[0] + w[1]*x1n) / w[2]
    x2_line = x2n * f2_std + f2_mean
    axes[0].plot(x1_line, x2_line, 'k--', lw=2)

axes[0].set_title(f'Logistic Regression 2F (Acc={acc_2f:.1%})')
axes[0].set_xlabel('Rotational speed [rpm]'); axes[0].set_ylabel('Torque [Nm]')
axes[0].legend(); axes[0].grid(alpha=0.3)

axes[1].plot(loss_hist_2f, 'b-', lw=1.5)
axes[1].set_title('BCE Loss'); axes[1].set_xlabel('epoch'); axes[1].grid(alpha=0.3)

plt.tight_layout(); plt.show()
print('\n-> 선형 결정 경계 하나로는 두 클래스를 잘 분리할 수 없습니다.')

---
# 2. Logistic Regression — 5 Features

변수를 **5개 전체**로 늘려 봅니다.  
같은 선형 모델이지만, feature가 많아지면 정확도가 개선됩니다.

In [ ]:
# 5 features 전체, 표준화
X5_mean = X_bal.mean(axis=0)
X5_std  = X_bal.std(axis=0)
X5_norm = (X_bal - X5_mean) / X5_std
X5 = np.column_stack([np.ones(len(y_bal)), X5_norm])   # (N, 6)

w5 = np.zeros(6)
rho = 0.01
n_epochs = 300

loss_hist_5f = []
for ep in range(n_epochs):
    o = sigmoid(X5 @ w5)
    loss = -np.sum(y_bal * np.log(o+1e-15) + (1-y_bal) * np.log(1-o+1e-15))
    loss_hist_5f.append(loss)
    grad = X5.T @ (o - y_bal)
    w5 = w5 - rho * grad

pred_5f = (sigmoid(X5 @ w5) >= 0.5).astype(int)
acc_5f = np.mean(pred_5f == y_bal)
print(f'Logistic (5 features) Accuracy: {acc_5f:.1%}')
print(f'\n  2 features: {acc_2f:.1%} → 5 features: {acc_5f:.1%}  (+{acc_5f-acc_2f:.1%})')

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(loss_hist_2f, 'b--', lw=1.5, label=f'2 features ({acc_2f:.1%})')
ax.plot(loss_hist_5f, 'r-',  lw=1.5, label=f'5 features ({acc_5f:.1%})')
ax.set_title('Logistic Regression: 2F vs 5F')
ax.set_xlabel('epoch'); ax.set_ylabel('BCE Loss')
ax.legend(); ax.grid(alpha=0.3)
plt.tight_layout(); plt.show()
print('-> Feature를 늘리면 정보량이 증가하여 정확도가 향상됩니다.')
print('   하지만 선형 모델의 한계로 ~83% 수준에서 정체됩니다.')

---
# 3. PyTorch MLP — 5 Features

동일한 5개 feature를 **다층 퍼셉트론(MLP)**으로 학습합니다.  
Practice 10에서 배운 PyTorch `nn.Sequential` + `Adam` 패턴을 사용합니다.

### 모델 구조

```
Input(5) → Linear(64) → ReLU → Linear(32) → ReLU → Linear(1) → Sigmoid
```

> **왜 MLP가 더 잘 될까?**  
> Logistic Regression은 하나의 초평면(hyperplane)으로 분류합니다.  
> MLP는 은닉층의 **비선형 변환**을 통해 복잡한 결정 경계를 학습할 수 있습니다.  
> (Practice 10의 XOR 문제에서 확인한 것과 동일한 원리입니다.)

In [ ]:
# PyTorch 텐서 변환
X_t = torch.tensor(X5_norm, dtype=torch.float32)   # (N, 5) — 표준화된 5 features
y_t = torch.tensor(y_bal,   dtype=torch.float32).unsqueeze(1)  # (N, 1)

# MLP 모델 정의
torch.manual_seed(42)
model = nn.Sequential(
    nn.Linear(5, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1)       # logit 출력 (BCEWithLogitsLoss가 sigmoid 적용)
)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(model)
n_params = sum(p.numel() for p in model.parameters())
print(f'\nTotal parameters: {n_params}')

In [ ]:
n_epochs = 500
loss_hist_mlp = []

for epoch in range(n_epochs):
    logits = model(X_t)
    loss = criterion(logits, y_t)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss_hist_mlp.append(loss.item())
    if (epoch + 1) % 100 == 0:
        with torch.no_grad():
            pred = (torch.sigmoid(model(X_t)) >= 0.5).int().flatten()
            acc = (pred == y_t.int().flatten()).float().mean()
        print(f'Epoch {epoch+1:3d}  Loss: {loss.item():.4f}  Acc: {acc:.1%}')

# 최종 정확도
with torch.no_grad():
    pred_mlp = (torch.sigmoid(model(X_t)) >= 0.5).int().flatten()
    acc_mlp = (pred_mlp == y_t.int().flatten()).float().mean().item()

print(f'\nMLP (5 features) Final Accuracy: {acc_mlp:.1%}')

In [ ]:
fig, axes = plt.subplots(1, 2)

# Loss curve (MLP만 — NumPy logistic은 total loss, PyTorch는 mean loss로 스케일이 다름)
axes[0].plot(loss_hist_mlp, 'g-', lw=1.5)
axes[0].set_title('MLP BCE Loss'); axes[0].set_xlabel('epoch'); axes[0].grid(alpha=0.3)

# 정확도 막대 그래프
models = ['Logistic\n2 feat', 'Logistic\n5 feat', 'MLP\n5 feat']
accs = [acc_2f, acc_5f, acc_mlp]
colors = ['#4488cc', '#cc4444', '#44aa44']
bars = axes[1].bar(models, [a*100 for a in accs], color=colors, edgecolor='k', width=0.5)
for bar, acc in zip(bars, accs):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                 f'{acc:.1%}', ha='center', va='bottom', fontweight='bold')
axes[1].set_title('Accuracy Comparison'); axes[1].set_ylabel('Accuracy (%)')
axes[1].set_ylim(0, 105); axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout(); plt.show()

---
# 요약

| | Logistic (2F) | Logistic (5F) | MLP (5F) |
|---|---|---|---|
| **Features** | RPM, Torque | 5개 전체 | 5개 전체 |
| **Model** | 선형 (NumPy) | 선형 (NumPy) | 비선형 (PyTorch) |
| **구조** | $\sigma(\mathbf{Xw})$ | $\sigma(\mathbf{Xw})$ | 5→64→32→1 (ReLU) |
| **Parameters** | 3 | 6 | ~2,500 |
| **Accuracy** | ~75% | ~83% | **~96%** |

### 배운 점

1. **Feature 수 증가** → 정보량 증가 → 정확도 향상 (75% → 83%)
2. **비선형 모델(MLP)** → 복잡한 결정 경계 학습 가능 → 정확도 극적 향상 (83% → 96%)
3. 실제 산업 데이터는 선형 분리가 어려운 경우가 많아, **MLP 이상의 모델**이 필요합니다

> Practice 06 (Extra)의 Steel Plates 데이터는 2개 feature만으로도 ~91%가 나왔지만,  
> AI4I 데이터는 고장 패턴이 복잡하여 선형 모델로는 한계가 분명했습니다.  
> **데이터의 특성에 따라 적절한 모델을 선택하는 것**이 중요합니다.